## Last weekend, I asked GPT4 from OpenAI to implement an association rule mining algorithm in Python. GPT-4 responded with the following:
`Association rule mining is a technique used in data mining to discover relationships between variables in large datasets. One popular algorithm for association rule mining is the Apriori algorithm. The following is a simple implementation of the Apriori algorithm in Python.`

In [22]:
from itertools import chain, combinations, filterfalse

# generates all possible subsets, then we use filterfalse
def old_powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1))



def join_set(itemsets, k):
    return set(
        [itemset1.union(itemset2) for itemset1 in itemsets for itemset2 in itemsets if len(itemset1.union(itemset2)) == k]
    )
###############################
def old_itemsets_support(transactions, itemsets, min_support):
    support_count = {itemset: 0 for itemset in itemsets}
    for transaction in transactions:
        for itemset in itemsets:
            if itemset.issubset(transaction):
                support_count[itemset] += 1
    n_transactions = len(transactions)
    x = {itemset: support / n_transactions for itemset, support in support_count.items() if support / n_transactions >= min_support}
    return x

def itemsets_support(transactions, itemsets, min_support):
    support_count = {itemset: 0 for itemset in itemsets}
    for transaction in transactions:
        for itemset in itemsets:
            if itemset <= transaction:  # Does the same as itemset.issubset(transaction) but faster
                support_count[itemset] += 1
    n_transactions = len(transactions)
    x = {itemset: support / n_transactions for itemset, support in support_count.items() if support / n_transactions >= min_support}
    return x
###############################

def old_apriori(transactions, min_support):
    items = set(chain(*transactions))
    itemsets = [frozenset([item]) for item in items]
    itemsets_by_length = [set()]
    k = 1
    while itemsets:
        support_count = itemsets_support(transactions, itemsets, min_support)
        itemsets_by_length.append(set(support_count.keys()))
        k += 1
        itemsets = join_set(itemsets, k)
    frequent_itemsets = set(chain(*itemsets_by_length))
    return frequent_itemsets, itemsets_by_length  

# no need to track itemsets by length
def apriori(transactions, min_support):
    items = set(chain(*transactions))
    itemsets = [frozenset([item]) for item in items]
    frequent_itemsets = set()
    k = 1
    while itemsets:
        support_count = itemsets_support(transactions, itemsets, min_support)
        frequent_itemsets.update(support_count.keys())
        k += 1
        itemsets = join_set(support_count.keys(), k)
    return frequent_itemsets

###############################

def old_association_rules(transactions, min_support, min_confidence):
    frequent_itemsets = apriori(transactions, min_support)
    rules = []
    for itemset in frequent_itemsets:
        for subset in filterfalse(lambda x: not x, powerset(itemset)): 
            antecedent = frozenset(subset)
            consequent = itemset - antecedent
            support_antecedent = len([t for t in transactions if antecedent.issubset(t)]) / len(transactions)
            support_itemset = len([t for t in transactions if itemset.issubset(t)]) / len(transactions)
            # calculate how many times the 
            confidence = support_itemset / support_antecedent
            if confidence >= min_confidence:
                rules.append((antecedent, consequent, support_itemset, confidence))
    return rules

# remove filterfalse as we did not generate empty sets
def association_rules(transactions, min_support, min_confidence):
    frequent_itemsets = apriori(transactions, min_support)
    rules = []
    for itemset in frequent_itemsets:
        for subset in powerset(itemset):
            antecedent = frozenset(subset)
            consequent = itemset - antecedent
            if consequent:  # Skip rules with empty consequent
                support_antecedent = len([t for t in transactions if antecedent <= t]) / len(transactions)
                support_itemset = len([t for t in transactions if itemset <= t]) / len(transactions)
                confidence = support_itemset / support_antecedent
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, support_itemset, confidence))
    return rules


In [31]:
# Test 1
transactions = [
    {"A", "B", "C"},
    {"A", "B"},
    {"A", "B"},
    {"B", "C"},
    {"B"},
    {"C","B"},
]

min_support = 0.2
min_confidence = 0.5
rules = association_rules(transactions, min_support, min_confidence)

if not rules:
    print("[]")

for antecedent, consequent, support, confidence in rules:
    print(f"{antecedent} => {consequent} (support={support:.2f}, confidence={confidence:.2f})")

frozenset({'B'}) => frozenset({'C'}) (support=0.50, confidence=0.50)
frozenset({'C'}) => frozenset({'B'}) (support=0.50, confidence=1.00)
frozenset({'A'}) => frozenset({'B'}) (support=0.50, confidence=1.00)
frozenset({'B'}) => frozenset({'A'}) (support=0.50, confidence=0.50)


In [32]:
# Example usage
transactions = [
    {"A", "B", "C"},
    {"A", "B"},
    {"A", "C"},
    {"A"},
    {"B", "C"},
    {"B"},
    {"C"},
]
min_support = 0.2  # only items that appeared min_support times will be considered
min_confidence = 0.5  # value between 0 and 1, 1 meaning that only rules with 100% confidence will be returned
rules = association_rules(transactions, min_support, min_confidence)
for antecedent, consequent, support, confidence in rules:
    print(f"{antecedent} => {consequent} (support={support:.2f}, confidence={confidence:.2f})")

frozenset({'B'}) => frozenset({'C'}) (support=0.29, confidence=0.50)
frozenset({'C'}) => frozenset({'B'}) (support=0.29, confidence=0.50)
frozenset({'A'}) => frozenset({'C'}) (support=0.29, confidence=0.50)
frozenset({'C'}) => frozenset({'A'}) (support=0.29, confidence=0.50)
frozenset({'A'}) => frozenset({'B'}) (support=0.29, confidence=0.50)
frozenset({'B'}) => frozenset({'A'}) (support=0.29, confidence=0.50)


In this implementation, we first define helper functions like `powerset`, `join_set`, and `itemsets_support` for finding power sets, joining itemsets, and calculating support values. Then, we implement the `apriori` function, which uses the Apriori algorithm to find frequent itemsets. Finally, we create the `association_rules` function to find the rules with minimum support and confidence.  

<span style="color:Red"> **TODO: TASK 1/3** </span> 
I am already impressed, although the algorithm isn't perfect. Please find and report the mistakes in the implementation, or tell me how it can be improved to correctly implement the Apriori algorithm (Task 1). 
- According to Bart, `support` and `confidence` are too high

===============================================================================
## Next, I asked:"How can I use association rules to build a recommendation algorithm? ", and it responded the following:  
Association rules can be used to build a simple recommendation algorithm by analyzing the relationships between items in a dataset, such as user-item interactions or item co-occurrences in transactions. By applying the Apriori algorithm or another association rule mining algorithm, you can extract rules with high support and confidence, indicating strong associations between items. These rules can then be used to generate recommendations.
Here's a step-by-step guide to building a recommendation algorithm using association rules:
1. **Prepare the dataset:** Organize your data into transactions or sets of items that are typically grouped together, such as items purchased together in a shopping cart or movies watched by a user.
2. **Apply association rule mining:** Use an algorithm like Apriori to extract frequent itemsets and generate association rules with high support and confidence. These rules represent patterns in the dataset that suggest items are frequently associated with each other.
3. **Build a recommendation function:** Create a function that takes a set of items as input, such as the items in a user's shopping cart or a user's watched movie list. `Use` the `association rules` to `find items` that are `strongly associated` `with` the `input items` `but not already in the input set`. `Sort` the `recommended items by` `confidence`, `support`, `or another metric`, and `return` the `top N items` as recommendations.
Here's a simple example of a recommendation function using the previously implemented Apriori algorithm:

In [41]:
def recommend_items(input_items, rules, recommendation_type: str = "average", top_n=5):
    # Initialize an empty dictionary to store the recommendations
    recommendations = {}

    # Iterate over each rule in the rules list
    for antecedent, consequent, support, confidence in rules:
        # Check if the antecedent is a subset of input_items and the consequent is not a subset of input_items
        if antecedent.issubset(input_items) and not consequent.issubset(input_items):
            # Iterate over each item in the consequent
            for item in consequent:
                # Check if the item is not already in the input_items
                if item not in input_items:
                    # If the item is not in recommendations, create an empty list for it
                    if item not in recommendations:
                        recommendations[item] = []
                    # Append the confidence and support values of the rule to the item's list in recommendations
                    recommendations[item].append((confidence, support))

    # METHOD 1: Calculate the average confidence and support for each recommended item
    def average_recommendations():
        avg_recommendations = {
            item: (sum(conf for conf, _ in item_rules) / len(item_rules),
                    sum(sup for _, sup in item_rules) / len(item_rules))
            for item, item_rules in recommendations.items()
        }
        return avg_recommendations

    # METHOD 2: Use maximum confidence as the recommendation score
    def max_confidence_recommendations():
        max_conf_recommendations = {
            item: max(scores, key=lambda x: x[0])
            for item, scores in recommendations.items()
        }
        return max_conf_recommendations

    # METHOD 3: Calculate lift for each recommended item
    def lift_recommendations():
        lift_recommendations = {
            item: max([(confidence / support, support) for confidence, support in item_rules])
            for item, item_rules in recommendations.items()
        }
        return lift_recommendations

    if recommendation_type == "average":
        recommendations = average_recommendations()
    elif recommendation_type == "max_confidence":
        recommendations = max_confidence_recommendations()
    elif recommendation_type == "lift":
        recommendations = lift_recommendations()

    # Sort the recommendations based on the chosen method in descending order
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][0], -x[1][1]))

    # Return the top n recommended items
    return [item for item, _ in sorted_recommendations[:top_n]]


In [44]:
# Here's how to use the `recommend_items` function:
input_items = {"A", "B"}
transactions = [
    {"A", "B", "C"},
    {"A", "B"},
    {"A", "C"},
    {"A"},
    {"B", "C"},
    {"B"},
    {"C"},
]
min_support = 0.2
min_confidence = 0.5
rules = association_rules(transactions, min_support, min_confidence)
recommended_items = recommend_items(input_items, rules)
print("Recommended items (Average):", recommended_items)

recommended_items = recommend_items(input_items, rules, recommendation_type="max_confidence")
print("Recommended items (Max_conf):", recommended_items)

recommended_items = recommend_items(input_items, rules, recommendation_type="lift")
print("Recommended items (Lift):", recommended_items)

Recommended items (Average): ['C']
Recommended items (Max_conf): ['C']
Recommended items (Lift): ['C']


In this example, given the input items {"A", "B"}, the recommendation function returns item "C" as a recommendation, since it has a strong association with items "A" and "B" based on the generated association rules.
Keep in mind that this is a simple recommendation algorithm, and more sophisticated methods such as collaborative filtering, matrix factorization, or deep learning models might yield better results depending on your dataset and use case.

## I asked to explain the rationale behind the recommend_items function, and recieved the following response:
Certainly! In the `recommend_items` function, we rank the items based on their association with the input items (antecedents). To do this, we calculate an aggregate score for each item using the confidence and support values of the association rules in which the item is present as a consequent. Here's a step-by-step breakdown of the ranking process:
1. **Initialize an empty dictionary for recommendations:** We start by creating an empty dictionary called `recommendations`. The keys in this dictionary will be the items we recommend, and the values will be lists of tuples containing the confidence and support values of the association rules in which the item is a consequent.
2. **Iterate through the association rules:** For each rule `(antecedent, consequent, support, confidence)`, we check if the `antecedent` is a `subset` ``of`` the `input items` and if the `consequent` is `not a subset` ``of`` the `input items`. This condition ensures that we only consider rules where the antecedents are present in the input items, and the consequents are not already included in the input items.
3. **Update the recommendations dictionary:** `If` the `condition` in step 2 is `met`, we `iterate` `through` the `items` `in` the `consequent`. `For` each `item` that is `not` already `in` the `input items`, we `add` an `entry` `to` the `recommendations` dictionary with the `item as` the `key` and the rule's `confidence` `and` `support` values `as` a tuple in the corresponding `value` list.
4. **Calculate the average confidence and support for each item:** We `create` a new `dictionary` that `contains` the `average confidence and support` for each recommended item by `summing` the `confidence and support` values of the corresponding rules and dividing by the number of rules in which the item appears as a consequent.
5. **Sort the recommended items:** We `sort` the recommended items `based` on their `average confidence and support` values in descending order. We `first` sort `by confidence`, and `then by support` as a tie-breaker. This is done using the `sorted` function with a custom sorting key.
6. **Return the top N recommended items:** We return the top N recommended items based on the sorted order. In this case, N is defined by the `top_n` parameter.
In summary, the `recommend_items` function ranks the items based on the average confidence and support values of the association rules in which the items appear as consequents. This ranking approach assumes that items with higher confidence and support values are more likely to be relevant and interesting to the user, given the input items.

## I also asked it to write an evaluation function:
To evaluate the quality of the `recommend_items` function, you can use measures like precision, recall, and F1 score. To compute these metrics, you'll need a test dataset with known user-item interactions. The evaluation function would calculate how well the recommendations generated by the `recommend_items` function match the known interactions in the test dataset.
Here's an example of an evaluation function that computes precision, recall, and F1 score for the `recommend_items` function:

In [33]:
"""
To use this evaluation function, you'll need a test dataset in the form of a dictionary where the keys represent user IDs, and the values are sets of true items for those users. You'll also need a dictionary called `user_items` that contains the input items for each user.
Please note that this evaluation function assumes the availability of ground truth data and that the recommendations generated by the `recommend_items` function are binary (i.e., an item is either relevant or not relevant). In practice, you may need to adapt the evaluation function to better suit your specific dataset and use case.
"""
def evaluate_recommendations(test_data, rules, top_n=5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for user, true_items in test_data.items():
        # Assuming user_items is a dictionary with user IDs as keys and their associated items as values
        input_items = user_items[user]
        # Get recommendations for the user
        recommended_items = set(recommend_items(input_items, rules, top_n=top_n))
        true_items = set(true_items)
        true_positives += len(recommended_items.intersection(true_items))
        false_positives += len(recommended_items - true_items)
        false_negatives += len(true_items - recommended_items)
    # Calculate precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1_score

<span style="color:Red"> **TODO: TASK 2/3** </span> 
I would like you to play around with this implementation and see if you can improve it already by using different ranking methods (in stead of average confidence). 

<span style="color:Red"> **TODO: TASK 3/3** </span> 
Last but not least, I would like you to use my implementation of Non-Derivable-Itemsets (http://www.adrem.ua.ac.be/~goethals/software/files/ndi.tgz) and check whether it generates better recommendations when only using association rules made of non-derivable itemsets (this will allow much smaller support thresholds!).  
(Unfortunately, GPT4 could not implement NDI)
- Please also experiment and report your results for different confidence thresholds.
- For experiments, you can use the retail dataset here: http://fimi.uantwerpen.be/data/ or any other dataset that you can find online for the evaluation of recommender systems. 


# TO SUBMIT:
- documented code and a script to run your tests (data included)
- a paper (PDF) in which you describe your findings.